I focus to transition of cnt month sales.

I divided transition of cnt month sales datast to each train data, and use LightGBM to predict future sales.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgb
from tqdm import tqdm
from itertools import product
from sklearn.model_selection import train_test_split
import seaborn as sns

import sys
import os
import gc
from glob import glob
import pickle
import json
import subprocess
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, train_test_split, RepeatedStratifiedKFold


In [ ]:
# !pip install --no-index --find-links ../input/pytorchset timm
# !pip install --no-index --find-links ../input/pytorchset pytorch-lightning

In [ ]:
# import torch
# import torch.optim as optim
# import torch.nn as nn
# import torch.nn.functional as F

# import torchvision.transforms as T
# import timm
# from sklearn.model_selection import StratifiedKFold
# from torchvision.io import read_image
# from torch.utils.data import DataLoader, Dataset

# import pytorch_lightning as pl
# from pytorch_lightning.utilities.seed import seed_everything
# from pytorch_lightning import callbacks
# from pytorch_lightning.callbacks.progress import ProgressBarBase
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning import LightningDataModule, LightningModule

# from sklearn.metrics import mean_squared_error
# from scipy.optimize import minimize

In [ ]:
# import tensorflow as tf
# import tensorflow_addons as tfa
# from tensorflow import keras
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# import lightgbm as lgb

# import cupy as cp # linear algebra
# import cudf as cd # data processing, CSV file I/O (e.g. pd.read_csv)

# from cuml.svm import SVR
# from cuml.decomposition import PCA

# from cuml.metrics import accuracy_score

In [ ]:
DF_CEILING_VALUE = 20000.0
SHOP_CEILING_VALUE = 999999.9
ITEM_CEILING_VALUE = 999999.9

TRANSITION_LENGTH = 18

In [ ]:
for i in range(0, 33 - TRANSITION_LENGTH):
    print(i)

In [ ]:
PATH = '../input/competitive-data-science-predict-future-sales/'

df = pd.read_csv(PATH + 'sales_train.csv')
df_test = pd.read_csv(PATH + 'test.csv')
sample = pd.read_csv(PATH + 'sample_submission.csv')
items = pd.read_csv(PATH + 'items.csv')
shops = pd.read_csv(PATH + 'shops.csv')
item_cats = pd.read_csv(PATH + 'item_categories.csv')

In [ ]:
data_files_names = ["df","df_test","sample","items","shops","item_cats"]
data_files = [df,df_test,sample,items,shops,item_cats]

Preprocessing each data

In [ ]:
df_test = df_test.drop('ID', axis=1)
df_test

In [ ]:
print(len(df[(df['item_cnt_day']>0.0)&(df['item_cnt_day']<10.0)]))
plt.hist(df[(df['item_cnt_day']>0.0)&(df['item_cnt_day']<10.0)]['item_cnt_day'],bins=20)

In [ ]:
df.loc[df['item_cnt_day'] < -1.0, 'item_cnt_day'] = -1.0
print(len(df[df['item_cnt_day'] < -1.0]))
df[df['item_cnt_day'] < 0.0]

In [ ]:
for i in range(3):
    df.loc[df['item_cnt_day'] > 10.0, 'item_cnt_day'] = df[df['item_cnt_day'] > 10.0]['item_cnt_day'] / 10.0
    df[df['item_cnt_day'] > 10.0]['item_cnt_day']
# df.loc[df['item_cnt_day'] > 10.0, 'item_cnt_day'] = 1.0

In [ ]:
len(df[df['item_cnt_day']>10.0])

In [ ]:
print(len(df[(df['item_cnt_day']>0.0)&(df['item_cnt_day']<10.0)]))
plt.hist(df[(df['item_cnt_day']>0.0)&(df['item_cnt_day']<10.0)]['item_cnt_day'],bins=20)

In [ ]:
df

In [ ]:
df = df.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False
        ).agg({'item_cnt_day':'sum'}
        ).rename(columns={'item_cnt_day':'mon_shop_item_cnt'})

In [ ]:
df

In [ ]:
# Preprocessing transition
def prepro_transition(transition, ceiling_value):
    p_transition = transition
    
    p_transition = p_transition.fillna(0)
    # p_transition = p_transition.drop('ID', axis=1)
    
    # Outliers to 10.0
#     p_transition[p_transition[0:34] > ceiling_value] = ceiling_value
    p_transition[p_transition.iloc[:, 2:] > ceiling_value] = ceiling_value
    
    return p_transition

In [ ]:
# left 214200
transition = df_test
for i in range(34):
    transition = pd.merge(transition, df[df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i }), how='left')
transition = prepro_transition(transition, DF_CEILING_VALUE)
transition

In [ ]:
# # Outer -> 526920
# transition = df_test
# for i in range(34):
#     transition = pd.merge(transition, df[df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i}), how='outer')
# transition = prepro_transition(transition, DF_CEILING_VALUE)
# transition

In [ ]:
transition.max()

In [ ]:
# Shop
shop_df = df.groupby(['date_block_num', 'shop_id'], as_index=False
        ).agg({'mon_shop_item_cnt':'sum'}
        )
shop_df

In [ ]:
# Shop
shop_transition = shops
for i in range(34):
    shop_transition = pd.merge(shop_transition, shop_df[shop_df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i}), on='shop_id', how='left')
shop_transition = shop_transition.fillna(0)
shop_transition

In [ ]:
shop_transition.mean()

In [ ]:
# Shop
item_df = df.groupby(['date_block_num', 'item_id'], as_index=False
        ).agg({'mon_shop_item_cnt':'sum'}
        )
item_df

In [ ]:
# Shop
item_transition = items
for i in range(34):
    item_transition = pd.merge(item_transition, item_df[item_df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i}), on='item_id', how='left')
item_transition = item_transition.fillna(0)
item_transition

In [ ]:
item_transition.mean()

In [ ]:
transition

In [ ]:
transition.max()

In [ ]:
transition = transition.drop([33], axis=1)
shop_transition = shop_transition.drop([33], axis=1)
item_transition = item_transition.drop([33], axis=1)

In [ ]:
transition

In [ ]:
%%time
transition_array = transition.to_numpy().tolist()
shop_transition_array = shop_transition.to_numpy().tolist()
item_transition_array = item_transition.to_numpy().tolist()

In [ ]:
pd.DataFrame(columns = ['shop_id', 'item_id'] + [x for x in range(TRANSITION_LENGTH)])

In [ ]:
%%time
x_train = []
y_train = []

for row in transition_array:
    for i in range(0, 33 - TRANSITION_LENGTH):
        if row[2+i+TRANSITION_LENGTH] < 20.0:
            row_shop = shop_transition_array[int(row[0])][2+i : 2+i+TRANSITION_LENGTH] if shop_transition_array[int(row[0])] else [0.0]*(TRANSITION_LENGTH)
            row_item = item_transition_array[int(row[1])][2+i : 2+i+TRANSITION_LENGTH] if item_transition_array[int(row[1])] else [0.0]*(TRANSITION_LENGTH)
            x_train.append(row[0:2] + row[2+i : 2+i+TRANSITION_LENGTH] + row_shop + row_item )
            y_train.append(row[2+i+TRANSITION_LENGTH] if row[2+i+TRANSITION_LENGTH] < DF_CEILING_VALUE else DF_CEILING_VALUE)
        
            
    

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
len(x_train[0])

In [ ]:
len(x_train)

In [ ]:
len(y_train)

In [ ]:
# # Set Callbacks
# def model_checkpoint(kind, fold):
#     return ModelCheckpoint(f'{kind}{fold:03}.h5',
#               verbose = 1, 
#               monitor = 'val_loss', 
#               mode = 'min', 
#               save_weights_only=True,
#               save_best_only = True)
# def early_stopping():
#     return EarlyStopping(
#         monitor='val_loss',
#         min_delta=0.0,
#         patience=EARRY_STOP,
#     )

# def create_model():
#     # Create Model
#     # base_model = SwinTransformer('swin_base_384', include_top=False, pretrained=True, use_tpu=True)
    
#     x_input = tf.keras.layers.Input(shape=(35,))

# #     x = tf.keras.layers.Dropout(0.1)(x)
#     x = tf.keras.layers.Dense(256, activation=tf.nn.relu)(x_input)
#     x = tf.keras.layers.Dense(256, activation=tf.nn.relu)(x)
#     x = tf.keras.layers.Dense(64, activation=tf.nn.relu)(x)
#     x = tf.keras.layers.Dense(64, activation=tf.nn.relu)(x)
#     x = tf.keras.layers.Dense(1, activation=tf.nn.relu)(x)

#     model = keras.Model(inputs = x_input, outputs = x)

#     model.compile(
#         optimizer = tfa.optimizers.AdamW(
#             learning_rate=LR, weight_decay=WD
#         ),
#         loss = tf.keras.losses.MeanAbsoluteError(),
# #         metrics=[tf.keras.metrics.RootMeanSquaredError()],
#     )

#     return model
# model = create_model()

In [ ]:
%%time
val_transition = df_test
row_i = 0
# plus 1 to get val_y
i = 0
for i in range(TRANSITION_LENGTH + 1):
    # subtraction from colum as y
    row_i = 34 - 1 - TRANSITION_LENGTH + i
    val_transition = pd.merge(val_transition, df[df['date_block_num']==row_i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i }), how='left')

y_val = val_transition[i].fillna(0)
val_transition = val_transition.drop([i], axis=1)

for i in range(TRANSITION_LENGTH):
    row_i = 34 - 1 - TRANSITION_LENGTH + i
    val_transition = pd.merge(val_transition, shop_df[shop_df['date_block_num']==row_i].drop(['date_block_num'], axis=1).rename(columns={'mon_shop_item_cnt': TRANSITION_LENGTH + i }), how='left')
for i in range(TRANSITION_LENGTH):
    row_i = 34 - 1 - TRANSITION_LENGTH + i
    val_transition = pd.merge(val_transition, item_df[item_df['date_block_num']==row_i].drop(['date_block_num'], axis=1).rename(columns={'mon_shop_item_cnt': TRANSITION_LENGTH * 2 + i }), how='left')


val_transition = val_transition.fillna(0)

val_transition

In [ ]:
val_transition = val_transition[y_val < 20.0]
y_val = y_val[y_val < 20.0]

In [ ]:
print(len(val_transition), len(y_val))

In [ ]:
y_val

In [ ]:
y_val.mean()

In [ ]:
y_val.max()

In [ ]:
lgb_train = lgb.Dataset(x_train, y_train, free_raw_data=False)
lgb_val = lgb.Dataset(val_transition, y_val, reference=lgb_train, free_raw_data=False)


lgbm_params = {
    'objective': 'mse',
    'metric': 'rmse',
    "num_leaves": 500,
    'is_unbalance':True,
    'boosting':'gbdt',
    "learning_rate": 0.01,
    'num_boost_round': 100000,
    'early_stopping_rounds':200
}




# # optimized by oputuna
# lgbm_params = {
#     'objective': 'mse',
#     'metric' : 'rmse',
#     "num_leaves": 966,
#     "cat_smooth": 45.01680827234465,
#     "min_child_samples": 27,
#     "min_child_weight": 0.021144950289224463,
#     "max_bin": 214,
#     "learning_rate": 0.01,
#     "subsample_for_bin": 300000,
#     "min_data_in_bin": 7,
#     "colsample_bytree": 0.8,
#     "subsample": 0.6,
#     "subsample_freq": 5,
#     "n_estimators": 3000,
# }


In [ ]:
model = lgb.train(lgbm_params,
                  lgb_train,
                  valid_names=['train', 'valid'],
                  valid_sets=[lgb_train, lgb_val],
                  early_stopping_rounds=20,
                  verbose_eval=100)

In [ ]:
TRANSITION_LENGTH

In [ ]:
shop_df

In [ ]:
submit_transition = df_test
for i in range(TRANSITION_LENGTH):
    row_i = 34 - TRANSITION_LENGTH + i
    submit_transition = pd.merge(submit_transition, df[df['date_block_num']==row_i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i }), how='left')
for i in range(TRANSITION_LENGTH):
    row_i = 34 - TRANSITION_LENGTH + i
    submit_transition = pd.merge(submit_transition, shop_df[shop_df['date_block_num']==row_i].drop(['date_block_num'], axis=1).rename(columns={'mon_shop_item_cnt': TRANSITION_LENGTH + i }), how='left')
for i in range(TRANSITION_LENGTH):
    row_i = 34 - TRANSITION_LENGTH + i
    submit_transition = pd.merge(submit_transition, item_df[item_df['date_block_num']==row_i].drop(['date_block_num'], axis=1).rename(columns={'mon_shop_item_cnt': TRANSITION_LENGTH * 2 + i }), how='left')

submit_transition = submit_transition.fillna(0)
# p_transition = p_transition.drop('ID', axis=1)

start_column = 2
submit_transition[submit_transition.iloc[:, start_column:start_column + TRANSITION_LENGTH] > DF_CEILING_VALUE] = DF_CEILING_VALUE

start_column += TRANSITION_LENGTH
submit_transition[submit_transition.iloc[:, start_column:start_column + TRANSITION_LENGTH] > SHOP_CEILING_VALUE] = SHOP_CEILING_VALUE

start_column += TRANSITION_LENGTH
submit_transition[submit_transition.iloc[:, start_column:start_column + TRANSITION_LENGTH] > ITEM_CEILING_VALUE] = ITEM_CEILING_VALUE


submit_transition

In [ ]:
submit_transition.loc[0:10, 25:35]

In [ ]:

# for row in transition_array:
#     for i in range(0, 34 - TRANSITION_LENGTH):
#         row_shop = shop_transition_array[int(row[0])][2+i : 2+i+TRANSITION_LENGTH] if shop_transition_array[int(row[0])] else [0.0]*(TRANSITION_LENGTH+1)
#         row_item = item_transition_array[int(row[1])][2+i : 2+i+TRANSITION_LENGTH] if item_transition_array[int(row[1])] else [0.0]*(TRANSITION_LENGTH+1)
#         x_train.append(row[0:2] + row[2+i : 2+i+TRANSITION_LENGTH] + row_shop + row_item )
#         y_train.append(row[2+i+TRANSITION_LENGTH] if row[2+i+TRANSITION_LENGTH] < DF_CEILING_VALUE else DF_CEILING_VALUE)

In [ ]:
preds = model.predict(submit_transition)

In [ ]:
sample['item_cnt_month'] = preds
sample.to_csv('submission.csv', index=False)

In [ ]:
sample.head(50)

In [ ]:
sample['item_cnt_month'].mean()

In [ ]:
sample['item_cnt_month'].std()

In [ ]:
sample['item_cnt_month'].max()

In [ ]:
plt.hist(sample['item_cnt_month'],bins=50)

In [ ]:
plt.hist(sample[(sample['item_cnt_month']>0.0)&(sample['item_cnt_month']<10.0)]['item_cnt_month'],bins=50)

In [ ]:
plt.hist(sample[(sample['item_cnt_month']>10.0)]['item_cnt_month'],bins=50)